In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
transform=transforms.Compose([
    transforms.Resize((256,256)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])
dataset=ImageFolder(root="C:\\Users\\dhruv\\Downloads\\archive (5)\\FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET\\train",transform =transform)
train_loader=DataLoader(dataset,batch_size=32,shuffle=True)

In [ ]:
from collections import Counter
label_counts = Counter()
for _, labels in train_loader:
    label_counts.update(labels.tolist())  
for label, count in label_counts.items():
    print(f"Label {label}: {count} images")


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomFireCNN(nn.Module):
    def __init__(self):
        super(CustomFireCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.global_pool = nn.AdaptiveAvgPool2d(1)  # Global average pooling

        self.fc1 = nn.Linear(512, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 3)  # 3 classes

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = F.relu(self.bn5(self.conv5(x)))

        x = self.global_pool(x)  # Convert to (batch, 512, 1, 1)
        x = torch.flatten(x, start_dim=1)  # Flatten for FC layers
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x


In [11]:
import torch
import torch.optim as optim
import torch.nn as nn


device = torch.device("cuda")
model = CustomFireCNN().to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 20 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels)  
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()

        # Display loss every 10 batches
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}")
            running_loss = 0.0  

Epoch [1/20], Batch [10/810], Loss: 0.7332
Epoch [1/20], Batch [20/810], Loss: 0.4047
Epoch [1/20], Batch [30/810], Loss: 0.4683
Epoch [1/20], Batch [40/810], Loss: 0.5322
Epoch [1/20], Batch [50/810], Loss: 0.5154
Epoch [1/20], Batch [60/810], Loss: 0.4944
Epoch [1/20], Batch [70/810], Loss: 0.4357
Epoch [1/20], Batch [80/810], Loss: 0.5522
Epoch [1/20], Batch [90/810], Loss: 0.4490
Epoch [1/20], Batch [100/810], Loss: 0.5700
Epoch [1/20], Batch [110/810], Loss: 0.4074
Epoch [1/20], Batch [120/810], Loss: 0.3791
Epoch [1/20], Batch [130/810], Loss: 0.3355
Epoch [1/20], Batch [140/810], Loss: 0.3348
Epoch [1/20], Batch [150/810], Loss: 0.3628
Epoch [1/20], Batch [160/810], Loss: 0.3926
Epoch [1/20], Batch [170/810], Loss: 0.3126
Epoch [1/20], Batch [180/810], Loss: 0.3381
Epoch [1/20], Batch [190/810], Loss: 0.3918
Epoch [1/20], Batch [200/810], Loss: 0.3447
Epoch [1/20], Batch [210/810], Loss: 0.2674
Epoch [1/20], Batch [220/810], Loss: 0.3366
Epoch [1/20], Batch [230/810], Loss: 0.27

KeyboardInterrupt: 

In [15]:
torch.save(model.state_dict(), "CustomCnn.pth")
print("Model saved successfully!")


Model saved successfully!


In [14]:

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


test_dataset = ImageFolder(root="C:\\Users\\dhruv\\Downloads\\archive (5)\\FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET\\test", transform=transform)


test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
model.eval()  # Set model to evaluation mode
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute loss
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)  # Get predicted labels
        correct += (predicted == labels).sum().item()
        total += labels.size(0)


avg_test_loss = test_loss / len(test_loader)
test_accuracy = (correct / total) * 100

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 0.1418, Test Accuracy: 96.09%


In [ ]:
import torch
import torchvision.transforms as transforms
from IPython.display import display
import torchvision.transforms.functional as TF


inv_normalize = transforms.Normalize(mean=[-0.5, -0.5, -0.5], std=[2, 2, 2])
dataiter = iter(train_loader)
images, labels = next(dataiter)

image_tensor = images[0]  # Shape: [C, H, W]

image_tensor = inv_normalize(image_tensor)
image_tensor = torch.clamp(image_tensor, 0, 1)  # Ensure values are within [0,1]

image_pil = TF.to_pil_image(image_tensor)
display(image_pil)


In [13]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

if not torch.cuda.is_available():
    raise SystemExit("CUDA is not available. Exiting...")

device = torch.device("cuda:0")
torch.cuda.set_device(device)  

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 3)  # 3 classes: Fire, Smoke, Non-Fire
model = model.to(device)  # Move model to GPU

assert next(model.parameters()).is_cuda, " Model is still on CPU! Something is wrong."

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # Print progress after every 10 batches
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1} completed. Avg Loss: {running_loss/len(train_loader):.4f}")



Epoch 1/10, Batch 0/810, Loss: 1.4428
Epoch 1/10, Batch 10/810, Loss: 0.3255
Epoch 1/10, Batch 20/810, Loss: 0.5548
Epoch 1/10, Batch 30/810, Loss: 0.0762
Epoch 1/10, Batch 40/810, Loss: 0.3143
Epoch 1/10, Batch 50/810, Loss: 0.1305
Epoch 1/10, Batch 60/810, Loss: 0.3013
Epoch 1/10, Batch 70/810, Loss: 0.1311
Epoch 1/10, Batch 80/810, Loss: 0.6567
Epoch 1/10, Batch 90/810, Loss: 0.2408
Epoch 1/10, Batch 100/810, Loss: 0.3449
Epoch 1/10, Batch 110/810, Loss: 0.1967
Epoch 1/10, Batch 120/810, Loss: 0.1254
Epoch 1/10, Batch 130/810, Loss: 0.3140
Epoch 1/10, Batch 140/810, Loss: 0.1459
Epoch 1/10, Batch 150/810, Loss: 0.0597
Epoch 1/10, Batch 160/810, Loss: 0.2532
Epoch 1/10, Batch 170/810, Loss: 0.3252
Epoch 1/10, Batch 180/810, Loss: 0.1034
Epoch 1/10, Batch 190/810, Loss: 0.3308
Epoch 1/10, Batch 200/810, Loss: 0.1434
Epoch 1/10, Batch 210/810, Loss: 0.3666
Epoch 1/10, Batch 220/810, Loss: 0.1478
Epoch 1/10, Batch 230/810, Loss: 0.0139
Epoch 1/10, Batch 240/810, Loss: 0.1043
Epoch 1/10,

In [16]:
correct = 0
total = 0
model.eval() 

with torch.no_grad():  
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")


Validation Accuracy: 98.16%


In [17]:
torch.save(model.state_dict(), "resnet_fire_detection.pth")
print("Model saved successfully!")


Model saved successfully!


In [18]:

test_dataset = ImageFolder(root="C:\\Users\\dhruv\\Downloads\\archive (5)\\FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET\\test", 
                           transform=transform)

# Create test DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluate on Test Set
correct = 0
total = 0
model.eval()  # Set model to evaluation mode

with torch.no_grad():  # No need to calculate gradients
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

# Print test accuracy
test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 98.40%
